# Analytic Plan Generator

This notebook uses artificial intelligence to generate analytic plans.

## Background

As described in TC 3-12.2.4.1, "The Analytic Scheme of Maneuver is the plan to collect and analyze technical data to meet specific information requirements. It identifies what data to analyze, how to analyze it, and why it is being analyzed." The analytic scheme of maneuver, or ASOM, consists of the following components:

* **Priority information requirement**:
* **Indicator**:
* **Evidence**:
* **Data**:
* **NAI**:
* **Analytic**:



## Environment Setup

This section sets up the environment. It installs packages necessary to generate the analytic plans, imports modules, initializes helper functions, and finally defines global variables. This section also mounts Google Drive to the runtime and moves into the project folder.

### Install Packages

In [1]:
!pip install -U -q "google" 1> /dev/null

### Import Modules

In [30]:
from google.colab import userdata
from google.colab import drive
import json
import os
import datetime
import pandas as pd
import re

### Initialize Helper Functions

In [11]:
def log(message, end="\n", flush = True):
    """
    Logs a message to the console, prepended with the current timestamp
    in ISO 8601 format.

    Args:
    message (str): The string message to log.
    """

    # Access the global flag controlling verbosity
    global verbose

    # Get the current date and time
    current_time = datetime.datetime.now()

    # Format the timestamp in ISO 8601 format
    timestamp = current_time.isoformat()

    # Construct the final log string using an f-string for clean formatting
    log_string = f"[{timestamp}] {message}"

    # Print the log string to the console if logging is turned on (verbose = True)
    if (verbose == True):
        print(log_string, end = end, flush = flush)

In [12]:
def build_asom(attack_chain):
    """
    Builds a list of JSON objects (asom) by processing technique files
    based on the provided attack_chain.

    Args:
        attack_chain (dict): A dictionary where keys are MITRE ATT&CK tactics
                             and values are lists of techniques.

    Returns:
        list: A list of JSON objects (asom) that match the criteria.
    """
    asom = []
    current_directory = os.getcwd() # Or specify a directory if not current

    for tactic, techniques in attack_chain.items():
        for technique in techniques:
            # Sanitize technique name to create a valid filename
            # Technique name might be "Txxxx - Technique Name"
            # Filename should be "Txxxx - Technique Name.json"
            file_name = f"{technique}.json"
            file_path = os.path.join(current_directory, file_name)

            if os.path.exists(file_path):
                try:
                    with open(file_path, 'r') as f:
                        technique_data_list = json.load(f)

                    # Ensure technique_data_list is a list
                    if not isinstance(technique_data_list, list):
                        print(f"Warning: Content of {file_name} is not a list. Skipping.")
                        continue

                    for item in technique_data_list:
                        # Ensure item is a dictionary and has at least one key
                        if not isinstance(item, dict) or not item:
                            print(f"Warning: Invalid item format in {file_name}. Skipping item: {item}")
                            continue

                        # The PIR is the first key in the item dictionary
                        pir_key = next(iter(item)) # Gets the first key

                        # Check if the parent tactic is in the PIR key
                        # The tactic in attack_chain is like "TA001 - Initial Access"
                        # We should check if "TA001" is in the pir_key
                        tactic_id = tactic.split(" - ")[0] # Extracts "TA001"
                        if tactic_id in pir_key:
                            asom.append(item)
                except json.JSONDecodeError:
                    print(f"Error decoding JSON from file: {file_name}")
                except Exception as e:
                    print(f"An error occurred while processing {file_name}: {e}")
            else:
                print(f"File not found for technique: {file_name}")
    return asom

### Define Global Variables

In [13]:
# Toggle logging on (verbose = True)/off (verbose = False)
verbose = True
# verbose = False

In [14]:
# Rate limits: https://ai.google.dev/gemini-api/docs/rate-limits
# Pricing: https://ai.google.dev/gemini-api/docs/pricing
# Usage: https://console.cloud.google.com/apis/api/generativelanguage.googleapis.com/metrics?project=gen-lang-client-0497172401
# Note that this notebook is designed to be run in Google Colab. The line below reads the Gemini API key for AI Studio,
# which is configured in the Secrets tab on the left side of the Colab window.
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")
log("Gemii API key loaded.")

[2025-05-08T10:49:34.622569] Gemii API key loaded.


### Mount Google Drive

In [15]:
# Mount Google Drive and move into the Google AI Studio folder
DRIVE_PATH = "/content/drive"
TECHNIQUES_PATH = "/content/drive/MyDrive/Google AI Studio/techniques"

drive.mount(DRIVE_PATH)
log(f"Google Drive mounted to {DRIVE_PATH}")

os.chdir(TECHNIQUES_PATH)
log(f"Changed directory to {TECHNIQUES_PATH}")

Mounted at /content/drive
[2025-05-08T10:49:51.877901] Google Drive mounted to /content/drive
[2025-05-08T10:49:52.676504] Changed directory to /content/drive/MyDrive/Google AI Studio/techniques


## ASOM Aggregation

This section first retrieves the latest MITRE ATT&CK Matrix for Enterprise from MITRE's GitHub repository. Then, it builds a data structure that stores each technique as they key with it's parent tactic, the technique's description, and detection suggestions as a nested dictionary as the value.

In [17]:
# Example attack_chain data structure
attack_chain_data = {
    "TA0001 - Initial Access": [
        "T1190 - Exploit Public-Facing Application",
        "T1566 - Phishing",
        "T1078 - Valid Accounts"
    ],
    "TA0004 - Privilege Escalation": [
        "T1078 - Valid Accounts"
    ]
}

log("Building ASOM...")
resulting_asom = build_asom(attack_chain_data)
log("Finished.")

# print("\nResulting ASOM:")
# if resulting_asom:
#     for entry in resulting_asom:
#         print(json.dumps(entry, indent=4))
# else:
#     log("ASOM is empty.")

[2025-05-08T10:54:30.970833] Building ASOM...
[2025-05-08T10:54:30.990119] Finished.


## ASOM Generation

In [28]:
def format_asom(asom_input_list):
    """
    Formats the ASOM data into a Pandas DataFrame with hierarchical indexing,
    including a column for the evidence description text.

    Args:
        asom_input_list (list): A list of JSON-like objects (dictionaries)
                                 as produced by the build_asom function.

    Returns:
        pandas.DataFrame: A DataFrame with the formatted data.
    """
    table_data = []
    pir_overall_index = 0

    for pir_object in asom_input_list:
        if not pir_object or not isinstance(pir_object, dict):
            print(f"Warning: Skipping invalid PIR object: {pir_object}")
            continue

        pir_overall_index += 1
        # Get the first key (PIR question)
        try:
            pir_text_key = next(iter(pir_object))
        except StopIteration:
            print(f"Warning: Skipping empty PIR object: {pir_object}")
            continue

        pir_content = pir_object[pir_text_key]

        if not isinstance(pir_content, dict):
            print(f"Warning: Skipping PIR object with invalid content for PIR key '{pir_text_key}'")
            continue

        indicators_section = pir_content.get("Indicators", {})
        technique_sub_index = 0

        if not isinstance(indicators_section, dict):
            print(f"Warning: 'Indicators' section is not a dictionary for PIR '{pir_text_key}'. Skipping indicators for this PIR.")
            # If indicators_section is empty or not a dict, no technique/evidence rows will be generated for this PIR.
            # If a row for the PIR itself is desired in such cases, additional logic would be needed here.
            if not indicators_section and pir_text_key: # Log if it's just empty but PIR exists
                print(f"Info: PIR '{pir_text_key}' has no 'Indicators' section or it's empty.")

        for technique_name, evidences_dict in indicators_section.items():
            technique_sub_index += 1
            current_technique_index_str = f"{pir_overall_index}.{technique_sub_index}"

            if not isinstance(evidences_dict, dict):
                print(f"Warning: Expected a dictionary of evidences for technique '{technique_name}' in PIR '{pir_text_key}', but got {type(evidences_dict)}. Skipping this technique's evidences.")
                # Optionally, create a row for the technique with blank evidence details
                # For now, this technique will not produce evidence rows.
                continue

            evidence_sub_index = 0
            if not evidences_dict: # If a technique has no evidence entries
                 print(f"Info: Technique '{technique_name}' has no evidence entries under PIR '{pir_text_key}'.")
                 # If a row for the technique itself is desired, add here.
                 # Example:
                 # table_data.append({
                 #    "PIR Index": pir_overall_index, "PIR": pir_text_key,
                 #    "Technique Index": current_technique_index_str, "Technique": technique_name,
                 #    "Evidence Index": pd.NA, "Evidence": "No specific evidence listed",
                 #    "Data": pd.NA, "Data Platform": pd.NA, "NAI": pd.NA, "Action": pd.NA
                 # })
                 pass # No evidence rows to add if not creating a placeholder

            for evidence_description_text, evidence_details in evidences_dict.items():
                evidence_sub_index += 1
                current_evidence_index_str = f"{current_technique_index_str}.{evidence_sub_index}"

                if not isinstance(evidence_details, dict):
                    print(f"Warning: Evidence details for '{evidence_description_text}' under technique '{technique_name}' is not a dictionary. Creating row with limited info.")
                    row = {
                        "PIR Index": pir_overall_index,
                        "PIR": pir_text_key,
                        "Tactic ID": pir_text_key.split(" - ")[0].split("(")[1],
                        "Technique Index": current_technique_index_str,
                        "Technique": technique_name,
                        "Technique ID": technique_name.split(" - ")[0],
                        "Evidence Index": current_evidence_index_str,
                        "Evidence": evidence_description_text + " (Error: Malformed details)",
                        "Data": pd.NA, # Using pandas NA for missing data
                        "Data Platform": pd.NA,
                        "NAI": pd.NA,
                        "Action": pd.NA
                    }
                else:
                    row = {
                        "PIR Index": pir_overall_index,
                        "PIR": pir_text_key,
                        "Tactic ID": pir_text_key.split(" - ")[0].split("(")[1],
                        "Technique Index": current_technique_index_str,
                        "Technique": technique_name,
                        "Technique ID": technique_name.split(" - ")[0],
                        "Evidence Index": current_evidence_index_str,
                        "Evidence": evidence_description_text, # This is the added column
                        "Data": evidence_details.get("Data", ""),
                        "Data Platform": evidence_details.get("Data Platform", ""),
                        "NAI": evidence_details.get("NAI", ""),
                        "Action": evidence_details.get("Action", "")
                    }
                table_data.append(row)

    df = pd.DataFrame(table_data)

    # Define the column order for the DataFrame
    column_order = [
        "PIR Index", "PIR", "Tactic ID", "Technique Index", "Technique", "Technique ID",
        "Evidence Index", "Evidence", "Data", "Data Platform", "NAI", "Action"
    ]

    # Ensure all columns are present in the DataFrame, adding them with NA if they are missing
    # This is important if table_data was empty, or if some rows missed keys.
    for col in column_order:
        if col not in df.columns:
            df[col] = pd.NA

    return df[column_order]

In [29]:
formatted_df = format_asom(resulting_asom)
print("Formatted DataFrame:")
# To display the full content of cells if they are long
with pd.option_context('display.max_rows', None,
                        'display.max_columns', None,
                        'display.width', 1000,
                        'display.max_colwidth', None):
    display(formatted_df)

Formatted DataFrame:


,PIR Index,PIR,Tactic ID,Technique Index,Technique,Technique ID,Evidence Index,Evidence,Data,Data Platform,NAI,Action
0,1,Has the adversary gained initial access to the network via exploiting a public-facing application? (TA0001 - Initial Access),TA0001,1.1,T1190 - Exploit Public-Facing Application,T1190,1.1.1,"Error messages or unusual entries in application/web server logs indicative of attempted exploitation (e.g., SQL injection errors, command execution attempts, path traversal).",Zeek http.log,Network devices,Insert site-specific NAI here,"Monitor Zeek http.log for unusual request patterns, error codes (e.g., 4xx, 5xx in unexpected volume/context), or suspicious strings in URI/headers/body (e.g., SQL keywords, command execution syntax). Use frequency analysis on request parameters or paths to identify anomalies."
1,1,Has the adversary gained initial access to the network via exploiting a public-facing application? (TA0001 - Initial Access),TA0001,1.1,T1190 - Exploit Public-Facing Application,T1190,1.1.2,"Alerts from Network Intrusion Detection Systems (NIDS) or Web Application Firewalls (WAF) flagging exploit patterns (e.g., known CVE exploits, injection attempts, unusual request structures).","Zeek signatures.log, Zeek conn.log",Network devices,Insert site-specific NAI here,Analyze Zeek signatures.log for alerts triggered by suspicious network traffic patterns targeting public-facing services. Correlate alerts with Zeek conn.log for associated sessions. Use correlation analysis to link alerts across multiple connections or source IPs.
2,2,Has the adversary gained initial access to the network via phishing? (TA0001 - Initial Access),TA0001,2.1,T1566 - Phishing,T1566,2.1.1,Email traffic containing links to known malicious sites or attachments detected as potentially malicious.,"Zeek smtp.log, Zeek http.log, Zeek conn.log",Network devices,Insert site-specific NAI here,Analyze Zeek smtp.log for emails containing URLs. Extract URLs and compare against threat intelligence feeds of known malicious sites. Analyze Zeek http.log/conn.log for connections originating from internal hosts to external IPs matching threat intel or associated with suspicious URLs found in emails. Use frequency analysis of URLs or entropy calculations on URL strings to identify unusual link patterns.
3,2,Has the adversary gained initial access to the network via phishing? (TA0001 - Initial Access),TA0001,2.1,T1566 - Phishing,T1566,2.1.2,"Emails with spoofed sender addresses or exhibiting anomalous header patterns (e.g., missing DKIM/SPF, unexpected mail routes).",Zeek smtp.log,Network devices,Insert site-specific NAI here,"Analyze Zeek smtp.log for email headers. Extract 'From', 'Return-Path', 'Received' headers. Check SPF and DKIM status if available in logs. Identify emails where the 'From' domain does not align with the 'Return-Path' or originating server, or where SPF/DKIM fail. Calculate sender domain entropy or use statistical analysis on sender behavior to flag anomalies."
4,3,Has the adversary gained initial access using valid accounts? (TA0001 - Initial Access),TA0001,3.1,T1078 - Valid Accounts,T1078,3.1.1,Successful login to an internal resource using a valid account from an unusual geographic location or new/rare source IP address.,"Windows Event ID 4624, Zeek conn.log","Servers, Network devices",Insert site-specific NAI here,Collect successful login events (WinEvent 4624) and network connections (Zeek conn.log). Extract source IP and destination resource. Geolocate source IPs. Identify logins from new or rare IPs/countries using frequency analysis or clustering of historical login locations. Investigate logins from flagged locations.
5,3,Has the adversary gained initial access using valid accounts? (TA0001 - Initial Access),TA0001,3.1,T1078 - Valid Accounts,T1078,3.1.2,"Successful login to external services (VPN, OWA, RDP) using valid credentials from an untrusted source.","Windows Event ID 4624, Zeek conn.log","Servers, Network devices",Inser

In [31]:
def find_common_suffix(strings_list):
    """Helper function to find the longest common suffix among a list of strings."""
    if not strings_list:
        return ""
    # Reverse all strings, find common prefix, then reverse back the result
    reversed_strings = [s[::-1] for s in strings_list]
    common_reversed_prefix = os.path.commonprefix(reversed_strings)
    return common_reversed_prefix[::-1]

def get_representative_pirs_by_tactic(df_with_pirs):
    """
    Groups PIRs by MITRE Tactic and generates a representative "average" PIR string for each group.

    Args:
        df_with_pirs (pd.DataFrame): DataFrame containing a 'PIR' column with the
                                     PIR strings (e.g., output from format_asom).

    Returns:
        dict: A dictionary mapping each Tactic ID (str) to its representative
              PIR string (str).
    """
    if 'PIR' not in df_with_pirs.columns:
        print("Error: DataFrame must contain a 'PIR' column.")
        return {}

    df = df_with_pirs.copy()

    # 1. Extract Tactic ID and Tactic Signature from PIR strings
    def extract_tactic_info(pir_string):
        if pd.isna(pir_string):
            return None, None
        # Regex to capture (TAXXXX - Tactic Name) or (TAXXXX)
        match_full = re.search(r"(\((TA\d{4})\s*-\s*([^)]+)\))", pir_string) # Captures (TAXXXX - Name)
        if match_full:
            return match_full.group(2), match_full.group(1) # Tactic ID, Full Signature

        match_simple = re.search(r"\((TA\d{4})\)", pir_string) # Captures (TAXXXX)
        if match_simple:
            return match_simple.group(1), match_simple.group(1) # Tactic ID, Full Signature (which is just ID part)
        return None, None

    df[['Tactic_ID', 'Tactic_Signature']] = df['PIR'].apply(
        lambda x: pd.Series(extract_tactic_info(x))
    )

    # Filter out rows where Tactic_ID couldn't be extracted
    df_valid_tactics = df.dropna(subset=['Tactic_ID', 'Tactic_Signature'])
    if df_valid_tactics.empty:
        print("No valid Tactic IDs found in PIR strings to group by.")
        return {}

    representative_pirs_map = {}
    grouped_by_tactic = df_valid_tactics.groupby('Tactic_ID')

    for tactic_id, group in grouped_by_tactic:
        unique_pirs = group['PIR'].unique().tolist()

        # Use the Tactic_Signature from the first entry in the group (should be consistent)
        # Or ensure consistency / pick the most common if they vary (though they shouldn't if grouped by ID)
        tactic_signature = group['Tactic_Signature'].iloc[0]

        if len(unique_pirs) == 1:
            representative_pirs_map[tactic_id] = unique_pirs[0]
            continue

        # Get the core question parts by removing the tactic signature
        core_questions = []
        for pir in unique_pirs:
            # Safely remove signature: find its last occurrence if somehow duplicated
            # For simplicity, assume it's at the end or clearly separable
            if tactic_signature and pir.endswith(tactic_signature):
                core_q = pir[:-len(tactic_signature)].strip()
            elif tactic_signature and tactic_signature in pir: # more general removal
                core_q = pir.replace(tactic_signature, "").strip()
            else: # PIR doesn't contain the expected signature (should be rare if extraction was good)
                core_q = pir.strip()
            core_questions.append(core_q)

        # Filter out any empty core questions that might result from faulty stripping
        core_questions = [q for q in core_questions if q]
        if not core_questions: # All PIRs were essentially just the tactic_signature
            representative_pirs_map[tactic_id] = tactic_signature
            continue

        # Generate the "average" core question
        if all(q == core_questions[0] for q in core_questions):
            avg_core_question_text = core_questions[0]
        else:
            common_prefix_text = os.path.commonprefix(core_questions).strip()
            common_suffix_text = find_common_suffix(core_questions).strip()
            placeholder = "[details vary]"

            if common_prefix_text and common_suffix_text:
                # Check if placeholder is meaningful (i.e., prefix and suffix don't cover everything)
                # And ensure they don't improperly overlap to create nonsensical strings
                if len(common_prefix_text) + len(common_suffix_text) < min(len(cq.strip()) for cq in core_questions if cq.strip()):
                    avg_core_question_text = f"{common_prefix_text} {placeholder} {common_suffix_text}"
                # Handle cases where prefix might include suffix, or very little variation exists
                elif common_prefix_text == common_suffix_text: # e.g. "X" and "X"
                     avg_core_question_text = f"{common_prefix_text}" # or with placeholder if it's just one common word
                elif common_prefix_text.endswith(common_suffix_text) and len(common_suffix_text) > 0 : # e.g. prefix "ABC", suffix "BC"
                     avg_core_question_text = f"{common_prefix_text} {placeholder}"
                else: # Default if both exist but don't fit cleaner patterns
                    avg_core_question_text = f"{common_prefix_text} {placeholder} {common_suffix_text}"
            elif common_prefix_text:
                avg_core_question_text = f"{common_prefix_text} {placeholder}"
            elif common_suffix_text:
                avg_core_question_text = f"{placeholder} {common_suffix_text}"
            else: # No significant common prefix or suffix
                avg_core_question_text = "Various distinct questions"

            # Clean up excessive spacing around placeholder or if it stands alone
            avg_core_question_text = re.sub(r'\s*\[details vary]\s*', f" {placeholder} ", avg_core_question_text).strip()
            if avg_core_question_text == placeholder: # If only placeholder remains
                 avg_core_question_text = "Various distinct questions"


        # Combine with the tactic signature
        final_representative_pir = f"{avg_core_question_text} {tactic_signature}".strip()
        representative_pirs_map[tactic_id] = final_representative_pir

    return representative_pirs_map

In [33]:
# Re-create a sample DataFrame similar to format_asom's output for demonstration
sample_data_for_pir_averaging = {
    'PIR Index': [1, 1, 2, 2, 3, 4, 4],
    'PIR': [
        "Has the adversary gained initial access to the network via exploiting a public-facing application? (TA0001 - Initial Access)",
        "Has the adversary gained initial access to the network via phishing? (TA0001 - Initial Access)",
        "Is the adversary escalating privileges by exploiting a vulnerability? (TA0004 - Privilege Escalation)",
        "Is the adversary escalating privileges using valid accounts? (TA0004 - Privilege Escalation)",
        "Has the adversary established Command and Control? (TA0011 - Command and Control)", # Single entry for this tactic
        "Does the adversary collect data from local system? (TA0009 - Collection)",
        "Does the adversary collect data from network shares? (TA0009 - Collection)",
    ],
    # ... other columns from format_asom output ...
    'Technique': ['T1190', 'T1566', 'T1068', 'T1078', 'T1071', 'T1005', 'T1135'],
    'Evidence Index': ['1.1.1', '1.2.1', '2.1.1', '2.2.1', '3.1.1', '4.1.1', '4.2.1'],
    'Evidence': ['Evidence A', 'Evidence B', 'Evidence C', 'Evidence D', 'Evidence E', 'Evidence F', 'Evidence G'],
    'Data': ['Data A', 'Data B', 'Data C', 'Data D', 'Data E', 'Data F', 'Data G'],
    'Data Platform': ['Platform A', 'Platform B', 'Platform C', 'Platform D', 'Platform E', 'Platform F', 'Platform G'],
    'NAI': ['NAI A', 'NAI B', 'NAI C', 'NAI D', 'NAI E', 'NAI F', 'NAI G'],
    'Action': ['Action A', 'Action B', 'Action C', 'Action D', 'Action E', 'Action F', 'Action G']
}
df_example = pd.DataFrame(sample_data_for_pir_averaging)

print("Original DataFrame sample:")
print(df_example[['Tactic_ID', 'PIR']].head() if 'Tactic_ID' in df_example.columns else df_example[['PIR']].head())


representative_pirs = get_representative_pirs_by_tactic(df_example)

print("\nRepresentative PIRs by Tactic ID:")
if representative_pirs:
    for tactic_id, pir_representation in representative_pirs.items():
        print(f"  {tactic_id}: {pir_representation}")
else:
    print("  No representative PIRs generated.")

Original DataFrame sample:
                                                 PIR
0  Has the adversary gained initial access to the...
1  Has the adversary gained initial access to the...
2  Is the adversary escalating privileges by expl...
3  Is the adversary escalating privileges using v...
4  Has the adversary established Command and Cont...

Representative PIRs by Tactic ID:
  TA0001: Has the adversary gained initial access to the network via [details vary] ? (TA0001 - Initial Access)
  TA0004: Is the adversary escalating privileges [details vary] ? (TA0004 - Privilege Escalation)
  TA0009: Does the adversary collect data from [details vary] ? (TA0009 - Collection)
  TA0011: Has the adversary established Command and Control? (TA0011 - Command and Control)


In [36]:
representative_pirs_map = get_representative_pirs_by_tactic(formatted_df)

In [37]:
def update_pirs_and_create_groups(original_df, representative_pirs_map):
    """
    Replaces PIR strings in a DataFrame with their generalized versions and
    then groups the DataFrame by these new PIRs.

    Args:
        original_df (pd.DataFrame): The DataFrame with an 'PIR' column
                                     containing original PIR strings.
        representative_pirs_map (dict): A dictionary mapping Tactic IDs (e.g., 'TA0001')
                                        to their generalized PIR strings.

    Returns:
        tuple: A tuple containing:
               - final_df (pd.DataFrame): The DataFrame with updated PIRs.
               - grouped_by_pir (pd.core.groupby.generic.DataFrameGroupBy):
                 The DataFrame grouped by the new generalized PIRs.
    """
    if 'PIR' not in original_df.columns:
        print("Error: DataFrame must contain an 'PIR' column.")
        return original_df, None

    final_df = original_df.copy()

    # Helper function to extract Tactic ID from a PIR string
    def extract_tactic_id(pir_string):
        if pd.isna(pir_string):
            return None
        # This regex aims to find (TAXXXX) within the string
        match = re.search(r"\((TA\d{4}).*?\)", pir_string)
        if match:
            return match.group(1) # Returns TAXXXX
        return None

    # 1. Extract Tactic ID from original PIRs to use for mapping
    final_df['Tactic_ID_for_mapping'] = final_df['PIR'].apply(extract_tactic_id)

    # 2. Replace the 'PIR' column with the generalized PIRs
    # The .map() function looks up each 'Tactic_ID_for_mapping' in the representative_pirs_map
    # .fillna(final_df['PIR']) ensures that if a Tactic ID doesn't have a corresponding
    # generalized PIR in the map, the original PIR string is retained.
    final_df['PIR'] = final_df['Tactic_ID_for_mapping'].map(representative_pirs_map).fillna(final_df['PIR'])

    # Clean up the temporary mapping column
    final_df = final_df.drop(columns=['Tactic_ID_for_mapping'])

    # 3. Group the updated DataFrame by the new (generalized) 'PIR' column
    grouped_by_pir = final_df.groupby('PIR')

    return final_df, grouped_by_pir


In [40]:
# Perform the update and grouping
final_df, grouped_results = update_pirs_and_create_groups(formatted_df, representative_pirs_map)

print("\nfinal_df with PIRs replaced (sample):")
# Sorting by PIR can help visualize the groups within the DataFrame itself
print(final_df.sort_values(by='PIR')[['PIR Index', 'PIR', 'Technique']].head(10))
print("-" * 50)

print("\nShowing groups based on generalized PIRs:")
if grouped_results:
    for pir_name, group_df in grouped_results:
        print(f"\nGeneralized PIR: {pir_name}")
        print(f"Number of original detailed rows in this group: {len(group_df)}")
        # Displaying relevant columns for the first few rows of each group
        print(group_df[['PIR Index', 'Technique', 'Evidence']].head(3))
        print("...")
else:
    print("Grouping could not be performed.")


final_df with PIRs replaced (sample):
   PIR Index                                                PIR  \
6          4  Has the adversary achieved privilege escalatio...   
7          4  Has the adversary achieved privilege escalatio...   
0          1  Has the adversary gained initial access [detai...   
1          1  Has the adversary gained initial access [detai...   
2          2  Has the adversary gained initial access [detai...   
3          2  Has the adversary gained initial access [detai...   
4          3  Has the adversary gained initial access [detai...   
5          3  Has the adversary gained initial access [detai...   

                                   Technique  
6                     T1078 - Valid Accounts  
7                     T1078 - Valid Accounts  
0  T1190 - Exploit Public-Facing Application  
1  T1190 - Exploit Public-Facing Application  
2                           T1566 - Phishing  
3                           T1566 - Phishing  
4                     T1078 

In [45]:
print("Final DataFrame:")
# To display the full content of cells if they are long
with pd.option_context('display.max_rows', None,
                        'display.max_columns', None,
                        'display.width', 1000,
                        'display.max_colwidth', None):
    display(final_df.sort_values(by='Tactic ID'))

Final DataFrame:


,PIR Index,PIR,Tactic ID,Technique Index,Technique,Technique ID,Evidence Index,Evidence,Data,Data Platform,NAI,Action
0,1,Has the adversary gained initial access [details vary] ? (TA0001 - Initial Access),TA0001,1.1,T1190 - Exploit Public-Facing Application,T1190,1.1.1,"Error messages or unusual entries in application/web server logs indicative of attempted exploitation (e.g., SQL injection errors, command execution attempts, path traversal).",Zeek http.log,Network devices,Insert site-specific NAI here,"Monitor Zeek http.log for unusual request patterns, error codes (e.g., 4xx, 5xx in unexpected volume/context), or suspicious strings in URI/headers/body (e.g., SQL keywords, command execution syntax). Use frequency analysis on request parameters or paths to identify anomalies."
1,1,Has the adversary gained initial access [details vary] ? (TA0001 - Initial Access),TA0001,1.1,T1190 - Exploit Public-Facing Application,T1190,1.1.2,"Alerts from Network Intrusion Detection Systems (NIDS) or Web Application Firewalls (WAF) flagging exploit patterns (e.g., known CVE exploits, injection attempts, unusual request structures).","Zeek signatures.log, Zeek conn.log",Network devices,Insert site-specific NAI here,Analyze Zeek signatures.log for alerts triggered by suspicious network traffic patterns targeting public-facing services. Correlate alerts with Zeek conn.log for associated sessions. Use correlation analysis to link alerts across multiple connections or source IPs.
2,2,Has the adversary gained initial access [details vary] ? (TA0001 - Initial Access),TA0001,2.1,T1566 - Phishing,T1566,2.1.1,Email traffic containing links to known malicious sites or attachments detected as potentially malicious.,"Zeek smtp.log, Zeek http.log, Zeek conn.log",Network devices,Insert site-specific NAI here,Analyze Zeek smtp.log for emails containing URLs. Extract URLs and compare against threat intelligence feeds of known malicious sites. Analyze Zeek http.log/conn.log for connections originating from internal hosts to external IPs matching threat intel or associated with suspicious URLs found in emails. Use frequency analysis of URLs or entropy calculations on URL strings to identify unusual link patterns.
3,2,Has the adversary gained initial access [details vary] ? (TA0001 - Initial Access),TA0001,2.1,T1566 - Phishing,T1566,2.1.2,"Emails with spoofed sender addresses or exhibiting anomalous header patterns (e.g., missing DKIM/SPF, unexpected mail routes).",Zeek smtp.log,Network devices,Insert site-specific NAI here,"Analyze Zeek smtp.log for email headers. Extract 'From', 'Return-Path', 'Received' headers. Check SPF and DKIM status if available in logs. Identify emails where the 'From' domain does not align with the 'Return-Path' or originating server, or where SPF/DKIM fail. Calculate sender domain entropy or use statistical analysis on sender behavior to flag anomalies."
4,3,Has the adversary gained initial access [details vary] ? (TA0001 - Initial Access),TA0001,3.1,T1078 - Valid Accounts,T1078,3.1.1,Successful login to an internal resource using a valid account from an unusual geographic location or new/rare source IP address.,"Windows Event ID 4624, Zeek conn.log","Servers, Network devices",Insert site-specific NAI here,Collect successful login events (WinEvent 4624) and network connections (Zeek conn.log). Extract source IP and destination resource. Geolocate source IPs. Identify logins from new or rare IPs/countries using frequency analysis or clustering of historical login locations. Investigate logins from flagged locations.
5,3,Has the adversary gained initial access [details vary] ? (TA0001 - Initial Access),TA0001,3.1,T1078 - Valid Accounts,T1078,3.1.2,"Successful login to external services (VPN, OWA, RDP) using valid credentials from an untrusted source.","Windows Event ID 4624, Zeek conn.log","Servers, Network devices",Insert site-specific NAI here,"Monitor successful logins to externally facing services (WinEvent 4624 on access gateways, Z

In [46]:
for pir_name, group_df in grouped_results:
    display(group_df)

,PIR Index,PIR,Tactic ID,Technique Index,Technique,Technique ID,Evidence Index,Evidence,Data,Data Platform,NAI,Action
6,4,Has the adversary achieved privilege escalatio...,TA0004,4.1,T1078 - Valid Accounts,T1078,4.1.1,Successful login or attempted login by a stand...,"Windows Event ID 4624, Windows Event ID 4688","Servers, Endpoints",Insert site-specific NAI here,Monitor successful login events (WinEvent 4624...
7,4,Has the adversary achieved privilege escalatio...,TA0004,4.1,T1078 - Valid Accounts,T1078,4.1.2,Multiple distinct valid accounts logged into t...,"Windows Event ID 4624, Windows Event ID 4648","Servers, Endpoints",Insert site-specific NAI here,Analyze successful login events (WinEvent 4624...


,PIR Index,PIR,Tactic ID,Technique Index,Technique,Technique ID,Evidence Index,Evidence,Data,Data Platform,NAI,Action
0,1,Has the adversary gained initial access [detai...,TA0001,1.1,T1190 - Exploit Public-Facing Application,T1190,1.1.1,Error messages or unusual entries in applicati...,Zeek http.log,Network devices,Insert site-specific NAI here,Monitor Zeek http.log for unusual request patt...
1,1,Has the adversary gained initial access [detai...,TA0001,1.1,T1190 - Exploit Public-Facing Application,T1190,1.1.2,Alerts from Network Intrusion Detection System...,"Zeek signatures.log, Zeek conn.log",Network devices,Insert site-specific NAI here,Analyze Zeek signatures.log for alerts trigger...
2,2,Has the adversary gained initial access [detai...,TA0001,2.1,T1566 - Phishing,T1566,2.1.1,Email traffic containing links to known malici...,"Zeek smtp.log, Zeek http.log, Zeek conn.log",Network devices,Insert site-specific NAI here,Analyze Zeek smtp.log for emails containing UR...
3,2,Has the adversary gained initial access [detai...,TA0001,2.1,T1566 - Phishing,T1566,2.1.2,Emails with spoofed sender addresses or exhibi...,Zeek smtp.log,Network devices,Insert site-specific NAI here,Analyze Zeek smtp.log for email headers. Extra...
4,3,Has the adversary gained initial access [detai...,TA0001,3.1,T1078 - Valid Accounts,T1078,3.1.1,Successful login to an internal resource using...,"Windows Event ID 4624, Zeek conn.log","Servers, Network devices",Insert site-specific NAI here,Collect successful login events (WinEvent 4624...
5,3,Has the adversary gained initial access [detai...,TA0001,3.1,T1078 - Valid Accounts,T1078,3.1.2,"Successful login to external services (VPN, OW...","Windows Event ID 4624, Zeek conn.log","Servers, Network devices",Insert site-specific NAI here,Monitor successful logins to externally facing...


In [53]:
def create_spanning_display_df(input_df):
    """
    Prepares the DataFrame for a display that mimics row spanning by setting a MultiIndex.
    It ensures 'Tactic ID' and 'Technique ID' columns exist by extracting them if needed.

    Args:
        input_df (pd.DataFrame): The DataFrame with a 'PIR' column containing
                                 generalized PIR strings, and other relevant columns like
                                 'PIR Index', 'Technique Index', 'Technique'.

    Returns:
        pd.DataFrame: A new DataFrame with a MultiIndex set, ready for display.
    """
    df = input_df.copy()

    # --- Step 1: Ensure 'Tactic ID' and 'Technique ID' columns exist ---
    if 'PIR' in df.columns and 'Tactic ID' not in df.columns:
        def extract_tactic_id(pir_string):
            if pd.isna(pir_string): return None
            # Extracts TAXXXX from strings like "...(TAXXXX - Name)" or "...(TAXXXX)"
            match = re.search(r"\((TA\d{4}).*?\)", pir_string)
            return match.group(1) if match else None
        df['Tactic ID'] = df['PIR'].apply(extract_tactic_id)

    if 'Technique' in df.columns and 'Technique ID' not in df.columns:
        def extract_technique_id(technique_string):
            if pd.isna(technique_string): return None
            # Extracts TXXXX or TXXXX.XXX from the beginning of the string
            match = re.match(r"(T\d{4}(?:\.\d{3})?)", technique_string)
            return match.group(1) if match else None
        df['Technique ID'] = df['Technique'].apply(extract_technique_id)

    # --- Step 2: Define columns for the MultiIndex ---
    # These columns will be used to create the hierarchical index for visual spanning.
    # The order determines the levels of the hierarchy.
    index_columns = [
        'PIR Index',        # As it appears in your example image
        'PIR',              # The generalized PIR string
        'Tactic ID',        # Extracted Tactic ID
        'Technique Index',
        'Technique',        # Full Technique string
        'Technique ID'      # Extracted Technique ID
    ]

    # Filter out any defined index_columns that might not actually exist in the df
    # (e.g., if a base column like 'Technique' was missing, 'Technique ID' wouldn't be created)
    existing_index_columns = [col for col in index_columns if col in df.columns]

    if not existing_index_columns:
        print("Warning: No suitable columns found to set as MultiIndex for spanning display.")
        return df # Return original df if no indexing can be performed

    # --- Step 3: Set the MultiIndex and sort ---
    # Ensure columns are in the desired order before setting index, then data columns
    data_columns = [col for col in df.columns if col not in existing_index_columns]
    # It's good practice to have a defined order
    df_reordered = df[existing_index_columns + data_columns]

    styled_df = df_reordered.set_index(existing_index_columns)
    styled_df = styled_df.sort_index() # Sorting is key for visual grouping

    return styled_df

In [54]:
# Create the DataFrame with visual spanning
df_with_spanning_look = create_spanning_display_df(final_df)

print("\nDataFrame with visual spanning (MultiIndex):")
# When you print this DataFrame, Pandas will typically render the MultiIndex
# in a way that groups identical higher-level index values.
# Using to_string() often helps visualize this in console.
with pd.option_context('display.max_rows', None, 'display.width', 1000, 'display.max_colwidth', 40):
    display(df_with_spanning_look)

# For Jupyter notebooks, simply displaying the DataFrame would often render well:
# display(df_with_spanning_look)


DataFrame with visual spanning (MultiIndex):


Evidence Index                                 Evidence                                     Data             Data Platform                            NAI                                   Action
PIR Index PIR                                      Tactic ID Technique Index Technique                                Technique ID                                                                                                                                                                                                   
1         Has the adversary gained initial acce... TA0001    1.1             T1190 - Exploit Public-Facing Applica... T1190                 1.1.1  Error messages or unusual entries in...                            Zeek http.log           Network devices  Insert site-specific NAI here  Monitor Zeek http.log for unusual re...
                                                                                                                      T1190                 1.1.2  Alerts from Network Intrusion Detect...       Zeek signatures.log, Zeek conn.log           Network devices  Insert site-specific NAI here  Analyze Zeek signatures.log for aler...
2         Has the adversary gained initial acce... TA0001    2.1             T1566 - Phishing                         T1566                 2.1.1  Email traffic containing links to kn...  Zeek smtp.log, Zeek http.log, Zeek c...           Network devices  Insert site-specific NAI here  Analyze Zeek smtp.log for emails con...
                                                                                                                      T1566                 2.1.2  Emails with spoofed sender addresses...                            Zeek smtp.log           Network devices  Insert site-specific NAI here  Analyze Zeek smtp.log for email head...
3         Has the adversary gained initial acce... TA0001    3.1             T1078 - Valid Accounts                   T1078                 3.1.1  Successful login to an internal reso...     Windows Event ID 4624, Zeek conn.log  Servers, Network devices  Insert site-specific NAI here  Collect successful login events (Win...
                                                                                                                      T1078                 3.1.2  Successful login to external service...     Windows Event ID 4624, Zeek conn.log  Servers, Network devices  Insert site-specific NAI here  Monitor successful logins to externa...
4         Has the adversary achieved privilege ... TA0004    4.1             T1078 - Valid Accounts                   T1078                 4.1.1  Successful login or attempted login ...  Windows Event ID 4624, Windows Event...        Servers, Endpoints  Insert site-specific NAI here  Monitor successful login events (Win...
                                                                                                                      T1078                 4.1.2  Multiple distinct valid accounts log...  Windows Event ID 4624, Windows Event...        Servers, Endpoints  Insert site-specific NAI here  Analyze successful login events (Win...